In [241]:
import sqlite3
from sqlite3 import Error
from datetime import datetime as dt

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# DP

In [242]:
def load_otodom():
    """Load csv files with defined column names"""
    
    data_ads_cols = ["date", "user_id", "ad_id", "category_id", "params"]
    data_replies_cols = ["date", "user_id", "ad_id", "mails", "phones"]
    data_segmentation_cols = ["user_id", "segment"]
    data_categories_cols = ["category_id", "category_name"]

    # here you can find information about the announcements
    data_ads_df = pd.read_csv("data/data_ads.csv", delimiter=";", names=data_ads_cols)
    # information about the response per advertisement per day
    data_replies_df = pd.read_csv("data/data_replies.csv", delimiter=";", names=data_replies_cols)
    # segmentation mapping for each user
    data_segments_df = pd.read_csv("data/data_segments.csv", delimiter=";", names=data_segmentation_cols)
    # mapping to category tree
    data_categories_df = pd.read_csv("data/data_categories.csv", delimiter=";", names=data_categories_cols)
    
    return [data_ads_df, data_replies_df, data_segments_df, data_categories_df]

In [243]:
def check_info(source):
    """Check columns type for each DataFrame"""
    
    print("Checking info: \n")
    
    for df in source:
        print (df.info(), "\n")

In [244]:
def cut_missing(source, column):
    """Cut rows with missing values from original source and make new df with only null values"""
    
    data_replies_df = source
    null_indices =  data_replies_df[data_replies_df["phones"].isnull()].index.tolist()

    # cutting nulls
    null_list = []
    for i in null_indices:
           null_list.append(data_replies_df.iloc[i])

    # dropping nulls
    not_null_replies = data_replies_df.drop(null_indices)
    not_null_replies.reset_index(drop=True, inplace=True)

    # new DataFrame with missing values
    data_replies_cols = ["date", "user_id", "ad_id", "mails", "phones"]
    null_replies = pd.DataFrame(null_list, columns=data_replies_cols)
    null_replies.reset_index(drop=True, inplace=True)
    
    # saving to csv
    null_replies.to_csv("data/null_replies.csv")
    not_null_replies.to_csv("data/not_null_replies.csv")
    
    return [null_replies, not_null_replies]

In [245]:
def load_replies():
    """Load csv files with null/not null replies"""
    
    #data_replies_cols = ["date", "user_id", "ad_id", "mails", "phones"]
    
    # segmentation mapping for each user
    null_replies = pd.read_csv("data/null_replies.csv")#, names=data_replies_cols)
    # mapping to category tree
    not_null_replies = pd.read_csv("data/not_null_replies.csv")#, names=data_replies_cols)
    
    return [null_replies, not_null_replies]

In [246]:
def check_missing(source, df_names, ):
    """ """
    
    for df, names in zip(source, df_names):
        print(f"Missing in {names} %\n",round(df.isnull().sum()/len(df)*100, 2),"\n")

In [247]:
def select_split(source):
    """ """
    
    # features
    X = source.iloc[:,0:4]
    # target
    y = source.iloc[:,4]
    
    # split into train=0.8, test=0.2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    
    return [X_train, X_test, y_train, y_test]

In [248]:
def best_pipe_model(X_train, X_test, y_train, y_test, to_pred):
    """ """
    
    to_pred.drop(["phones"], axis=1, inplace=True)
    splits = [X_train, X_test, to_pred]
    
    for split in splits:
        #split["date"] = pd.to_datetime(split["date"])
        
        split["date"] = pd.to_datetime(split["date"])
        split["date"] = split["date"].apply(lambda x: x.toordinal())

    
    
    # pipeline
    pipe = Pipeline([("regressor", LinearRegression())])
    
    search_space = [
    {"regressor": [LinearRegression()]},
    {"regressor": [Ridge()],
    "regressor__alpha": np.linspace(0, 0.2, 21),
    "regressor__max_iter": [1000]},
    {"regressor": [Lasso()],
    "regressor__alpha": np.linspace(0, 0.2, 21),
    "regressor__max_iter": [1000]}
]
    # gridsearch, fit, predict
    gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=1, n_jobs=-1)
    best_model = gridsearch.fit(X_train, y_train)
    preds = best_model.predict(to_pred)
     
    return preds

In [249]:
def join_replies(source_1, source_2, pred):
    """ """
    
    # join DataFrames
    replies_1 = source_1
    replies_2 = source_2
    replies_2["phones"] = pred
    data_replies = replies_1.append(replies_2, ignore_index=True)
    
    return data_replies

In [250]:
def dp_job():
    # List of OLX DataFrames
    data_ads_df, data_replies_df, data_segments_df, data_categories_df = load_otodom()
    # Returns: [data_ads_df, data_replies_df, data_segments_df, data_categories_df]
    
    
#     # DataFrame: data_replies_df
#     data_replies_df = data[1]
    
#     # Names of splitted DataFrames
    
    
    # Check info
    #check_info(source=data)
    # Returns: None
    
     # Try load files from csv or make new
    try:
        null_replies, not_null_replies = load_replies()
    except Exception as e:
           print("Error has occurred: ", e, "\n")
    finally:
        # Split original source into two pieces [null/not null] ans save it to csv
        null_replies, not_null_replies = cut_missing(source=data_replies_df, column="phones")
        # Returns: [null_replies, not_null_replies]

        
    print("len null replies:", len(null_replies))
    print("len not null replies:", len(not_null_replies))
    
    # Select features, target and split it
    X_train, X_test, y_train, y_test = select_split(source=not_null_replies)
    # Retruns:  [X_train, X_test, y_train, y_test]
    
    splitted = [X_train, X_test, y_train, y_test]
    split_names = ["X_train", "X_test", "X_val", "y_train", "y_test", "y_val"]
    # Check percent of missing values 
    check_missing(source=splitted, df_names=split_names)
    # Returns: None
    
    # Classification WIP
    preds = best_pipe_model(X_train, X_test, y_train, y_test, to_pred=null_replies)
    # Returns: preds
    print("len preds:", len(preds))
    
    new_data_replies = join_replies(source_1=not_null_replies, source_2=null_replies, pred=preds)
    # Returns data_replies
    
    # Update list of DataFrames
    data_replies_df = new_data_replies
    
    return [data_ads_df, data_replies_df, data_segments_df, data_categories_df]

# SQL

In [251]:
def sqlite3_connect(db_file):
    """Establish connection with local database"""
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("Connected to sqlite3 ver: ", sqlite3.version)
    except Error as e:
        print(e)
        
    return conn

In [252]:
def sqlite3_insert(source, tables, connection):
    """Insert data to the database from DataFrames"""
    
    # [data_ads_df, data_replies_df, data_segments_df, data_categories_df]
    indices = [0, 1, 2, 3]
    
    for i, tab in zip(indices, tables):
        source[i].to_sql(tab, connection, if_exists='replace', index=False)


In [253]:
def sqlite3_query(connection):#, query):
    """Create a table from table_query statement"""
    
#     try:
#         connection.execute(query)
#         print("Query send!")
#     except Error as e:
#         print(e)
        
        
    sql = "SELECT * FROM replies WHERE phones >  40;"
    response_df = pd.read_sql_query(sql, connection)
    return response_df.head()

In [254]:
def sql_job(source):
    database = r"otodom.db"
    table_list = ["ads", "replies", "segments", "categories"]
    sql_my_query = " "
    
    conn = sqlite3_connect(database)
    
    if conn is not None:
        sqlite3_insert(source=source, tables=table_list, connection=conn)
        query = sqlite3_query(connection=conn)#, query=sql_my_query)
        print(query)
        #sqlite3_query(connection=conn, query=sql_my_query)
        #sqlite3_query(connection=conn, query=sql_my_query)
        print("SQL job is done.")
    else:
        print("Error! cannot create the database connection.")

# LIQUIDITY

In [255]:
def liquidity_per_user():
    """
    Liquidity will be understood as % of advertisements which have received 
    at least 1 response (by phone or e-mail) within a period of 7 days 
    (including day 0 - the day of adding an day of adding an ad)
    """
    
    pass

In [256]:
def full_data_analysis():
    """ 
    Jupyter/R Markdown preferred for analysis

    Scripts can be in separate files, or as part of a notebook depending on
    selected methods

    Please present your final results and most important conclusions in the 
    form of a presentation (e.g. Google slides)
    """
    
    pass

In [257]:
def question_1():
    """ 
    What differences do you see between the segments in terms of the data 
    you have available (including liquidity)?
    """
    
    pass

In [258]:
def question_2():
    """What do you think might influence higher or lower levels of liquidity?"""
    
    pass

# MAIN

In [259]:
def main():
    
    # Predict missing values with LinearRegression
    data = dp_job()
    
    # Make db, insert data from df
    sql_job(source=data)

In [260]:
if __name__ == '__main__':
    %time main()

len null replies: 110268
len not null replies: 643009
Missing in X_train %
 date       0.0
user_id    0.0
ad_id      0.0
mails      0.0
dtype: float64 

Missing in X_test %
 date       0.0
user_id    0.0
ad_id      0.0
mails      0.0
dtype: float64 

Missing in X_val %
 0.0 

Missing in y_train %
 0.0 

Fitting 5 folds for each of 43 candidates, totalling 215 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 215 out of 215 | elapsed:   31.8s finished
C:\Users\DJaskulski\miniconda3\envs\olx_otodom\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.51535e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


len preds: 110268
Connected to sqlite3 ver:  2.6.0
         date  user_id     ad_id  mails  phones
0  2019-04-14  2435764  58744356      0    48.0
1  2019-04-14  2435764  58743964      0    51.0
2  2019-04-14  2435764  58744404      0    72.0
SQL job is done.
Wall time: 1min 11s
